In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import pickle
from itertools import product 
from IPython.display import display
#import brightway2 as bw

### 1. define data structure, here using GT example, can expand shape to include more vehiles
    1: type	        
    2. powertrain   
    2.A: SIZE       
    3. v_year	        
    4. SSP	       
    5. Module	   
    6. Tech

In [2]:
tech = pd.read_excel("../data/tech_list/techlist.xlsx", sheet_name = "gtruck") #, index_col=[0,1])
tech.head()

,Module,Tech,LCIA_perunit,TRL_SSP585_2030,TRL_SSP119_2030,TRL_SSP585_2040,TRL_SSP119_2040,TRL_SSP585_2050,TRL_SSP119_2050
0,V1A,dummy_tech,kilogram,9,9,9,9,9,9
1,V1B,LIB_NMC622,kWh,9,9,9,9,9,9
2,V1B,LIB_NCA,kWh,9,9,9,9,9,9
3,V1B,LIB_LFP,kWh,9,9,9,9,9,9
4,V1B,LIB_LTO,kWh,9,9,9,9,9,9


In [3]:
len(tech["Tech"])

10

## SSP to investigate

FAIR (available scn https://docs.google.com/document/d/1oeZa6h3tMZcZE3mIjdAQ3cvVddpgJL9sSDg8Jma0AJw/edit )  
- "ssp119": "SSP1-1.9", (SSP1-RCP1.9 ~1.3 °C) 
- "ssp126": "SSP1-2.6",
- "ssp245": "SSP2-4.5",
- "ssp370": "SSP3-7.0",
- "ssp434": "SSP4-3.4",
- "ssp460": "SSP4-6.0",
- "ssp534-over": "SSP5-3.4-overshoot",
- "ssp585": "SSP5-8.5":: SSP5 baseline (which corresponds to RCP8.5)



Premise / :: name@remind: 

https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios

- ssp119 -> ~1.3 °C ::  SSP1-PkBudg500
- ssp126 -> ~1.7 °C ::  SSP1-PkBudg1150
- ssp245 -> ~3.5 °C ::  SSP2-Base
- <font color = 'red'> fair do not have other SSP2 scn:::  e.g., ssp119 or ssp226 </font>
- ssp370 -> N/A premise
- ssp585 -> ~4.5 °C ::  SSP5-Base

In [4]:
ssp_list = ["ssp119", "ssp126",  "ssp245", "ssp585"]  

In [5]:
year = [2030, 2040, 2050]

In [6]:
powertrain = ['ICEV-d', 'BEV']

In [7]:
size_GVWR =  [ "HDV_class7" ] #the GT curb weight is ~17ton, for the case study we only use one default size 

In [8]:
v_list = ['garbage_truck']

## Prepare DF and xarray: 
- layer 1: SSP
- layer 2: year
- layer 3: powertrain
- layer 4: size 
- layer 5: v_type

each layer DF len(i)  = len(i-1) * ... 

In [9]:
list_ssp = [] 
for ssp in ssp_list: 
    list_ssp.append( [ssp] * len(tech["Tech"]))

In [10]:
list_year = [] 
for yr in year: 
    list_year.append( [yr] *  (len(list_ssp[0]) * len(list_ssp)) )

In [11]:
list_pt = []
for pt in powertrain: 
    list_pt.append( [pt] * (len(list_year[0]) * len(list_year))  )

In [12]:
list_size = []
for sz in size_GVWR: 
    list_size.append( [sz] * (len(list_pt[0]) * len(list_pt))  )

In [13]:
list_vh = []
for vt in v_list: 
    list_vh.append([vt] * len(list_size[0]) * len(list_size) )

In [14]:
print( (len(list_vh[0]) * len(list_vh)) ,  (len(list_size[0]) * len(list_size)),  (len(list_pt[0]) * len(list_pt)),  (len(list_year[0]) * len(list_year)) ,  (len(list_ssp[0]) * len(list_ssp))  )

240 240 240 120 40


##### for each layer (other than final layer v_type), multiply len() of previous layers [layer[i] / layer[i-1]] to prepare final DF

In [15]:
list_size_f = [item for sublist in list_size for item in sublist]  
len_size = int((len(list_vh[0]) * len(list_vh)) / (len(list_size[0]) * len(list_size))) 
list_size2 = [list_size_f] * len_size
len(list_size2[0]) * len(list_size2) == (len(list_vh[0]) * len(list_vh))

True

In [16]:
list_pt_f = [item for sublist in list_pt for item in sublist]  
len_pt = len_size *  int((len(list_size[0]) * len(list_size)) / (len(list_pt[0]) * len(list_pt))) 
list_pt2 = [list_pt_f] * len_pt
len(list_pt2[0]) * len(list_pt2) == (len(list_vh[0]) * len(list_vh))

True

In [17]:
list_yr_f = [item for sublist in list_year for item in sublist]  
len_yr = len_pt *  int((len(list_pt[0]) * len(list_pt)) / (len(list_year[0]) * len(list_year)))
list_yr2 = [list_yr_f] *  len_yr
len(list_yr2[0]) * len(list_yr2) == (len(list_vh[0]) * len(list_vh))

True

In [18]:
list_ssp_f = [item for sublist in list_ssp for item in sublist]  
len_ssp = len_yr * int( (len(list_year[0]) * len(list_year)) / ( len(list_ssp) * len(tech["Tech"])) )
list_ssp2 = [list_ssp_f] *   len_ssp 
len(list_ssp2[0]) * len(list_ssp2) == (len(list_vh[0]) * len(list_vh))

True

In [19]:
mod2 = [tech["Module"].tolist()] * int((len(list_vh[0]) * len(list_vh)) /  (len(tech["Module"])))

In [20]:
tech2 = [tech["Tech"].tolist()] * int((len(list_vh[0]) * len(list_vh)) /  (len(tech["Module"])))

#### get final pandas DF, then convert to xarray

In [21]:
def flatten_list(lst):
    return [item for sublist in lst for item in (flatten_list(sublist) if isinstance(sublist, list) else [sublist])]

In [22]:
techf, modf, sspf, yrf, ptf, sizef, vhf = flatten_list(tech2), flatten_list(mod2), flatten_list(list_ssp2), flatten_list(list_yr2), flatten_list(list_pt2), flatten_list(list_size2), flatten_list(list_vh)

In [23]:
finaldf = pd.DataFrame(zip(vhf, sizef, ptf, yrf, sspf, modf, techf), columns=['v_type', 'size', 'powertrain', 'v_year', 'SSP', 'Module', 'Tech'])

In [24]:
finaldf

,v_type,size,powertrain,v_year,SSP,Module,Tech
0,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1A,dummy_tech
1,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NMC622
2,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_NCA
3,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LFP
4,garbage_truck,HDV_class7,ICEV-d,2030,ssp119,V1B,LIB_LTO
...,...,...,...,...,...,...,...
235,garbage_truck,HDV_class7,BEV,2050,ssp585,V1B,LSB
236,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_diesel,diesel_mix
237,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,REC_100PV
238,garbage_truck,HDV_class7,BEV,2050,ssp585,F2_fuel_elec,grid_elec


In [25]:
finaldf.to_excel("../data/to_use_interm/TRL_1A_tech_toaddTRL_wo_merged_header.xlsx")

In [26]:
# Make 'IndexColumn' as the index colume, and merge index
finaldf.set_index(['v_type', 'size', 'powertrain', 'v_year', 'SSP', 'Module', 'Tech'], inplace=True)

In [27]:
finaldf

Empty DataFrame
Columns: []
Index: [(garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, F2_fuel_diesel, diesel_mix), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, F2_fuel_elec, REC_100PV), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, F2_fuel_elec, grid_elec), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp119, V2_oper, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, F2_fuel_diesel, diesel_mix), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, F2_fuel_elec, REC_100PV), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, F2_fuel_elec, grid_elec), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp126, V2_oper, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, F2_fuel_diesel, diesel_mix), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, F2_fuel_elec, REC_100PV), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, F2_fuel_elec, grid_elec), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp245, V2_oper, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, F2_fuel_diesel, diesel_mix), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, F2_fuel_elec, REC_100PV), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, F2_fuel_elec, grid_elec), (garbage_truck, HDV_class7, ICEV-d, 2030, ssp585, V2_oper, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, F2_fuel_diesel, diesel_mix), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, F2_fuel_elec, REC_100PV), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, F2_fuel_elec, grid_elec), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp119, V2_oper, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1A, dummy_tech), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1B, LIB_NMC622), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1B, LIB_NCA), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1B, LIB_LFP), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1B, LIB_LTO), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, V1B, LSB), (garbage_truck, HDV_class7, ICEV-d, 2040, ssp126, F2_fuel_diesel, diesel_mix), (garbage_truck, HD

In [28]:
finaldf.to_excel("../data/to_use_interm/TRL_1A_tech_toaddTRL_merged_header.xlsx")

In [29]:
dtxr = finaldf.to_xarray()
dtxr

<xarray.Dataset>
Dimensions:     (v_type: 1, size: 1, powertrain: 2, v_year: 3, SSP: 4,
                 Module: 5, Tech: 9)
Coordinates:
  * v_type      (v_type) object 'garbage_truck'
  * size        (size) object 'HDV_class7'
  * powertrain  (powertrain) object 'BEV' 'ICEV-d'
  * v_year      (v_year) int64 2030 2040 2050
  * SSP         (SSP) object 'ssp119' 'ssp126' 'ssp245' 'ssp585'
  * Module      (Module) object 'F2_fuel_diesel' 'F2_fuel_elec' ... 'V2_oper'
  * Tech        (Tech) object 'LIB_LFP' 'LIB_LTO' ... 'dummy_tech' 'grid_elec'
Data variables:
    *empty*

In [30]:
dtxr.to_netcdf('../data/to_use_interm/TRL_1A_empty_veh_ds.nc')